In [ ]:
pip install ngboost scikit-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.1 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=3d52e4fb57ce89c1c20182bb517e978446085bbd50d61e312f76b5e48a339597
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [ ]:
pip install lightgbm optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.9 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from joblib import dump, load


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df = pd.read_csv('PCA_df_w_game_markersUPDATED.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
import pandas as pd

# Assuming 'Date' is your column containing the date in datetime format
df['gameDate'] = pd.to_datetime(df['gameDate'])  # Ensure the 'Date' column is in datetime format
target = 'win_indicator'
# Define the threshold date (July 1st, 2022)
threshold_date = pd.Timestamp('2022-07-01')

# Split the data into training and testing sets
train_df = df[df['gameDate'] < threshold_date]  # Data before July 1st, 2022 for training
test_df = df[df['gameDate'] >= threshold_date]  # Data after July 1st, 2022 for testing

# Separate features and target for training
X_train = train_df.drop(columns=[target, 'win_indicator'])  # Exclude target and Date columns
y_train = train_df[target]

# Separate features and target for testing
X_test = test_df.drop(columns=[target, 'win_indicator'])  # Exclude target and Date columns
y_test = test_df[target]

# Save the training and test sets to CSV files (optional)
train_df.to_csv("train_data_NHL.csv", index=False)
test_df.to_csv("test_data_NHL.csv", index=False)

# Display a few rows of each set for confirmation
print("Training Data:")
print(train_df.head())
print(len(train_df))
print("\nTest Data:")
print(test_df.head())
print(len(test_df))


In [ ]:
X_train.head()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
X_train = X_train.drop(['season', 'name', 'opposingTeam', 'Game_Number', 'gameDate'], axis=1)
X_test = X_test.drop(['season', 'name', 'opposingTeam', 'Game_Number', 'gameDate'], axis=1)

In [ ]:
X_train.head()

In [ ]:
train_df.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from joblib import dump

# Define hyperparameter grid
param_grid = {
    "n_estimators": [50, 75, 100, 125],  # Number of trees
    "max_depth": [10, 15, 20, 25],  # Maximum depth of trees
    "min_samples_split": [10, 15, 20, 30],  # Minimum samples required to split a node
    "min_samples_leaf": [5, 10, 12, 14, 16],  # Minimum samples per leaf node
    "max_features": ["sqrt", "log2"]  # Number of features per split
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Perform Randomized Search with 5-fold cross-validation
rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=20,  # Number of random combinations to try
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1,  # Use all available processors
    random_state=42
)

# Fit the model with the best parameters
rf_search.fit(X_train, y_train)
best_rf = rf_search.best_estimator_

# Make predictions
y_pred_train = best_rf.predict(X_train)
y_pred_test = best_rf.predict(X_test)

# Model evaluation
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='binary')
recall_test = recall_score(y_test, y_pred_test, average='binary')
f1_test = f1_score(y_test, y_pred_test, average='binary')
conf_matrix = confusion_matrix(y_test, y_pred_test)
class_report = classification_report(y_test, y_pred_test)

# Print results
print("Best Parameters:", rf_search.best_params_)
print("\nTrain Performance:")
print(f"  Accuracy: {accuracy_train:.4f}")
print("\nTest Performance:")
print(f"  Accuracy: {accuracy_test:.4f}")
print(f"  Precision: {precision_test:.4f}")
print(f"  Recall: {recall_test:.4f}")
print(f"  F1 Score: {f1_test:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Add predictions to DataFrame
#df["Predicted_wins"] = best_rf.predict(X)

dump(best_rf, '/content/drive/My Drive/SLMg77/Models/RF_model_thesis_Classification.joblib')


In [ ]:
dump(best_rf, '/content/drive/My Drive/SLMg77/Models/RF_model_thesis_Classification.joblib')

In [ ]:
df.head(30)


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from joblib import dump

# Define XGBoost model
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [3, 4],
    'learning_rate': [0.1, 0.01, 0.001, 0.05],
    'subsample': [0.6, 0.75, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# GridSearch for hyperparameter tuning
grid_search_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

# Fit the model using the existing X_train, y_train
grid_search_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)

# Get the best model
best_xgb_model = grid_search_xgb.best_estimator_

# Predictions
y_pred_train_xgb = best_xgb_model.predict(X_train)
y_pred_test_xgb = best_xgb_model.predict(X_test)

# Model evaluation
accuracy_train_xgb = accuracy_score(y_train, y_pred_train_xgb)
accuracy_test_xgb = accuracy_score(y_test, y_pred_test_xgb)
precision_test_xgb = precision_score(y_test, y_pred_test_xgb, average='binary')
recall_test_xgb = recall_score(y_test, y_pred_test_xgb, average='binary')
f1_test_xgb = f1_score(y_test, y_pred_test_xgb, average='binary')
conf_matrix_xgb = confusion_matrix(y_test, y_pred_test_xgb)
class_report_xgb = classification_report(y_test, y_pred_test_xgb)

# Print results
print("Best Parameters:", grid_search_xgb.best_params_)
print("\nTrain Performance:")
print(f"  Accuracy: {accuracy_train_xgb:.4f}")
print("\nTest Performance:")
print(f"  Accuracy: {accuracy_test_xgb:.4f}")
print(f"  Precision: {precision_test_xgb:.4f}")
print(f"  Recall: {recall_test_xgb:.4f}")
print(f"  F1 Score: {f1_test_xgb:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix_xgb)
print("\nClassification Report:")
print(class_report_xgb)

# Save the best model
dump(best_xgb_model, '/content/drive/My Drive/SLMg77/Models/XGB_model_thesis_Classification.joblib')

print("\nXGBoost model saved")


In [ ]:
X_train.head()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from ngboost import NGBClassifier
from ngboost.distns import Bernoulli  # Use Bernoulli for classification
from joblib import dump

# Define the model (with Bernoulli for classification)
ngb = NGBClassifier(Dist=Bernoulli)

# Define parameter grid for classification
param_grid = {
    'n_estimators': [100, 150, 200, 250],
    'learning_rate': [0.01, 0.05, 0.1],
    'minibatch_frac': [0.5, 0.8, 1.0],
    'natural_gradient': [True, False]
}

# Grid search with classification metrics
grid_search = GridSearchCV(ngb, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Best model predictions
best_ngb = grid_search.best_estimator_
y_pred_train = best_ngb.predict(X_train)
y_pred_test = best_ngb.predict(X_test)

# Evaluate model
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='binary')
recall_test = recall_score(y_test, y_pred_test, average='binary')
f1_test = f1_score(y_test, y_pred_test, average='binary')
conf_matrix = confusion_matrix(y_test, y_pred_test)
class_report = classification_report(y_test, y_pred_test)

# Print results
print("\nTrain Performance:")
print(f"  Accuracy: {accuracy_train:.4f}")
print("\nTest Performance:")
print(f"  Accuracy: {accuracy_test:.4f}")
print(f"  Precision: {precision_test:.4f}")
print(f"  Recall: {recall_test:.4f}")
print(f"  F1 Score: {f1_test:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


dump(best_ngb, '/content/drive/My Drive/SLMg77/Models/NGB_model_thesis_Classification.joblib')


print("\nModel saved")




In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from joblib import dump

# Define MLPClassifier
mlp = MLPClassifier(max_iter=200, random_state=42)

# Define hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (64, 32), (128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],  # L2 regularization
    'learning_rate': ['constant', 'adaptive'],
    'batch_size': [32, 64]
}

# GridSearchCV
grid_search_mlp = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

# Fit the model
grid_search_mlp.fit(X_train, y_train)

# Best model
best_mlp_model = grid_search_mlp.best_estimator_

# Predictions
y_pred_train = best_mlp_model.predict(X_train)
y_pred_test = best_mlp_model.predict(X_test)

# Evaluation
print("Best Parameters:", grid_search_mlp.best_params_)
print(f"\nTrain Accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_test):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

# Save the model
dump(best_mlp_model, '/content/drive/My Drive/SLMg77/Models/MLP_model_thesis_Classification.joblib')
print("\nMLP model saved.")
